Download the independency packages

In [0]:
!wget -q https://archive.apache.org/dist/spark/spark-2.1.1/spark-2.1.1-bin-hadoop2.7.tgz

In [0]:
!tar -zxf spark-2.1.1-bin-hadoop2.7.tgz

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!pip install -q findspark

Set up the environmental variables

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.1.1-bin-hadoop2.7"

Initialize Spark

In [0]:
import findspark
findspark.init()

Create a spark session and reading the text file

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
data=spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t')

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [0]:
from pyspark.sql.functions import length

In [0]:
data=data.withColumn('length', length(data['text']))

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [15]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



Feature Transformations

In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [0]:
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

In [0]:
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')

In [0]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [0]:
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

form the feature vector for ML

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

Model

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb=NaiveBayes()

Pipeline

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data)

In [0]:
clean_data=cleaner.transform(data)

Train and evaluate the data

In [0]:
clean_data=clean_data.select(['label','features'])

In [31]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [0]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [0]:
spam_predictor = nb.fit(training)

In [0]:
test_results = spam_predictor.transform(testing)

In [35]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,6,16,...|[-1000.6978135125...|[1.0,2.8170579935...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-881.57010655906...|[1.0,1.6479730749...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-870.98966006301...|[1.0,4.4454339596...|       0.0|
|  0.0|(13459,[0,1,25,66...|[-1343.9766402665...|[1.0,4.6833317808...|       0.0|
|  0.0|(13459,[0,1,29,38...|[-1492.4575067481...|[0.99999942950083...|       0.0|
|  0.0|(13459,[0,1,29,89...|[-1548.7929104344...|[0.99998792121941...|       0.0|
|  0.0|(13459,[0,1,33,46...|[-375.96390448472...|[1.0,3.0882407587...|       0.0|
|  0.0|(13459,[0,1,49,18...|[-1143.6298669216...|[7.23614867770573...|       1.0|
|  0.0|(13459,[0,1,439,6...|[-301.88405143550...|[0.99999999999999...|       0.0|
|  0.0|(13459,[0

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9243219878469574
